# Importing data

In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv') 
ids = test["PassengerId"]
dataset.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
def cleanData(data):
    data = data.drop(["Ticket","Cabin","Name","PassengerId"],axis=1)
    columns = ["SibSp","Parch","Age","Fare"]
    for col in columns:
        if(col == "Fare"):
            data[col].fillna(data[col].mean(), inplace =True)
        else:
        	data[col].fillna(data[col].median(), inplace =True)
    data.Embarked.fillna("U", inplace= True)
    return data

In [4]:
dataset = cleanData(dataset)
test = cleanData(test)

In [5]:
dataset.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


# converting categorical to numerical

In [6]:

def fillData(data,test):
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    columns = ["Embarked","Sex"]
    for col in columns:
        data[col] = le.fit_transform(data[col])
        test[col]  =le.transform(test[col])
    print(le.classes_)
    return data,test

dataset,test= fillData(dataset,test)


['female' 'male']


In [7]:
y = dataset["Survived"]
x = dataset.drop("Survived",axis=1)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)


In [8]:
print(x)

[[ 0.82737724  0.73769513 -0.56573646 ... -0.47367361 -0.50244517
   0.58111394]
 [-1.56610693 -1.35557354  0.66386103 ... -0.47367361  0.78684529
  -1.93846038]
 [ 0.82737724 -1.35557354 -0.25833709 ... -0.47367361 -0.48885426
   0.58111394]
 ...
 [ 0.82737724 -1.35557354 -0.1046374  ...  2.00893337 -0.17626324
   0.58111394]
 [-1.56610693  0.73769513 -0.25833709 ... -0.47367361 -0.04438104
  -1.93846038]
 [ 0.82737724  0.73769513  0.20276197 ... -0.47367361 -0.49237783
  -0.67867322]]


In [9]:
#print(test)

     Pclass  Sex   Age  SibSp  Parch      Fare  Embarked
0         3    1  34.5      0      0    7.8292         1
1         3    0  47.0      1      0    7.0000         2
2         2    1  62.0      0      0    9.6875         1
3         3    1  27.0      0      0    8.6625         2
4         3    0  22.0      1      1   12.2875         2
..      ...  ...   ...    ...    ...       ...       ...
413       3    1  27.0      0      0    8.0500         2
414       1    0  39.0      0      0  108.9000         0
415       3    1  38.5      0      0    7.2500         2
416       3    1  27.0      0      0    8.0500         2
417       3    1  27.0      1      1   22.3583         0

[418 rows x 7 columns]


In [10]:
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)
classifier = SVC(kernel = 'rbf', random_state = 0,C=0.75,gamma=0.1)
classifier.fit(x_train,y_train)

SVC(C=0.75, gamma=0.1, random_state=0)

In [11]:
y_pred = classifier.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8251121076233184

In [13]:
submission_pred = classifier.predict(sc.fit_transform(test)) 

In [14]:
df = pd.DataFrame({"PassengerId":ids.values,
                  "Survived":submission_pred})

In [15]:
df.to_csv("submission.csv",index=False)